In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher as SM
import pandas as pd
import numpy as np
import time
import Levenshtein
import re
from newsapi import NewsApiClient
import requests
import bs4 as bs
from bs4.element import Comment
from IPython.display import display,HTML
from newsapi import NewsApiClient

In [ ]:
#!pip install newsapi-python

In [14]:
data = pd.read_csv('dataXData.csv')
company = data['Business Name'].unique()
company = company.astype(str)
#data.head()

In [15]:
pd.unique(data[['City']].values.ravel())

array([nan, 'quebec', 'bc', ..., 'temperance', 'delmont', 'monaca'],
      dtype=object)

In [16]:
def nameCleaner(name): 
    name = name.lower()
    name = name.split()
    #eng_stopwords = set(stopwords.words("english"))
    #name = [w for w in name if not w in eng_stopwords]
    company_stopwords = set(['group', 'company', 'limited', 'cos', 'dba', 'lllp', 'ltd', 'sa', 'ag', 'pllc', 'pl', 'nv', 'ltcc', 'bv', 'gmblt', 'llc', 'sia', 'sta', 'inc', 'co', 'corp', 'pte'])
    a= []
    for w in name:
        w = re.sub('[^a-zA-Z0-9]+', '',w)
        if(w not in company_stopwords):
            a.append(w)
    name = ''.join(a)
    return name

In [17]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [23]:
start=time.time()
response='y'
while(response=='y'):
    a=[]
    text2=input("Which text do you want to search? ")
    text1 = nameCleaner(text2)
    company = data['Search Term'].unique()
    company = company.astype(str)
    for i in range(0,len(company),1):
        search=company[i].lower()
        a.append(fuzz.token_sort_ratio(text1,search))

    #Finding the company names with maximum risk score aka fuzzy ratio
    ratio=np.array(a)
    c=np.where(ratio==ratio.max())
    maximum=max(a)
    top = company[c]
    top_list = set(data.loc[data['Search Term'].isin(top)]['Business Name'])
    print("\nRisk Score for", color.BOLD +text2 + color.END,"is",color.BOLD + str(maximum) + color.END,"for the brands",top_list)
    
    #Decision Logic for Classification
    if maximum>=90:
        print(color.BOLD,color.RED +"High risk"+ color.END)
    elif (maximum<90 and maximum>=60):
        print(color.BOLD,color.YELLOW +"Medium Risk"+ color.END)
    else:
        print(color.BOLD,color.GREEN +"No Risk"+ color.END)

    # News API part begins here
    df=data.loc[data['Search Term'].isin(top)][['Business Name','Industry','City','State']]
    df1=df.drop_duplicates(keep='first')
    display(df1)
    unique_names = []
    for d in top_list:
        lower = d.lower()
        if lower not in unique_names:
            unique_names.append(lower)

    #print(unique_names)

    def text_from_html(body):
        soup = bs.BeautifulSoup(body, 'html.parser')
        texts = soup.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        # encode to get rid of encoding errors (hopefully)
        return " ".join(t.strip() for t in visible_texts).encode('utf-8','ignore')

    def tag_visible(element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True

    entity_column = []
    url_column = []
    snippet_column = []
    # Init
    newsapi = NewsApiClient(api_key='a28acbbe0b2a46428906f353d180c5c3')
    all_articles = []
    for entity in unique_names:
        all_articles=newsapi.get_everything(q=entity,
                                             language='en', # necessary to only maintain english? probably for now
                                             sort_by='relevancy',
                                             page=1)["articles"]

        for article in all_articles:
            url = article["url"]        
            source = requests.get(url)
            if source.status_code != 200:
                continue
            text = text_from_html(source.content).decode('utf-8')

            if (' ' not in entity):
                splitted = text.split()
                lower = [x.lower() for x in splitted]

                try:
                    index = lower.index(entity);

                except ValueError:
                    index = -1
                if(index != -1):
                    if(index < 50):
                        article_snippet = " ".join(splitted[0:index+50])
                    else:
                        article_snippet = " ".join(splitted[index-50:index+50])
                    entity_column.append(entity)
                    url_column.append(url)
                    snippet_column.append(article_snippet)
            else:
                indices = [m.start() for m in re.finditer(entity.lower(), text.lower())]

                if not indices: 
                    continue
                index = indices[0]
                if(index < 100):
                    article_snippet = text[0:index+100]
                else:
                    article_snippet = text[index-100:index+100]
                entity_column.append(entity)
                url_column.append(url)
                snippet_column.append(article_snippet)


    columns = {"Entity" : entity_column, "URL": url_column, "Snippet" : snippet_column}
    df_result = pd.DataFrame.from_dict(columns)
    pd.set_option('max_colwidth', 300)
    if (df_result.empty):
        print(color.BOLD + "Sorry, No Results found on Internet" + color.END)
    else:
        display(df_result) 
    
    #Recording time taken to search the database and NEWS API
    end=time.time()
    print("\nTime taken:",round(end-start,0),"seconds")
    response=input("\nDo you want to search again? [y/n] ")
print(color.BOLD + "\nTHANK YOU!, for searching with us!" + color.END)

Which text do you want to search? john

Risk Score for john is 100 for the brands {'JOHN'}
 High risk


,Business Name,Industry,City,State
633887,JOHN,Lessors of real estate (including mini warehouses & self-storage units),los angeles,CA


,Entity,URL,Snippet
0,john,https://www.nytimes.com/2018/11/10/us/politics/john-delaney-2020-iowa.html,"Sections SEARCH Skip to content Skip to site index Subscribe Log In Politics | John Delaney Is Already Running for President Image John Delaney, a Congressman from Maryland, spoke at the Democratic Wing Ding in Clear Lake, Iowa, in August. Credit Credit George Etheredge for The New York Times Jo..."
1,john,https://www.cnn.com/videos/tv/2018/11/27/csr-john-wayne-gacy-ron-2.cnn,Store How To Watch VR Archives More… Photos Longform Investigations CNN profiles CNN Leadership CNN Newsletters Work for CNN U.S. International Español Arabic Set edition preference: U.S. International Confirm Facebook Twitter Instagram Chat with us in Facebook Messenger. Find out what's happeni...
2,john,https://www.cnn.com/videos/politics/2018/11/05/john-oliver-last-week-tonight-trump-family-separation-lisa-france-elizabeth-prann-ots-vpx.hln,Breaking News 45 Congress Supreme Court 2018 Election Results Search » Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds. JUST WATCHED John Oliver slams Trump over family separation Replay More Videos ... MUST WATCH John Oliver slams Trump over family separ...
3,john,https://www.cnn.com/videos/entertainment/2018/11/29/john-legend-chrissy-teigen-holiday-special-sot-chloe-melas-ots-vpx.hln,Breaking News Search Stars Screen Binge Culture Media Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds. JUST WATCHED A look at Legend and Teigen's holiday special Replay More Videos ... MUST WATCH A look at Legend and Teigen's holiday special Celebrity cou...
4,john,https://www.nytimes.com/2018/11/09/style/john-leguizamo-has-harsh-words-for-us.html,"Sections SEARCH Skip to content Skip to site index Style Log In Subscribe Log In Today’s Paper Style | John Leguizamo Has Harsh Words for Us Supported by El Espace John Leguizamo Has Harsh Words for Us This week in El Espace: J. Lo, the midterms, punk rock and brown ballet shoes. Image In his ne..."
5,john,https://www.nytimes.com/2018/11/25/opinion/will-mitch-run.html,"Subscribe Log In Today’s Paper Opinion | Will Mitch Run? Opinion Supported by Will Mitch Run? Landrieu speaks with certainty on many issues, but a 2020 candidacy may not yet be one of them. By Charles M. Blow Opinion Columnist Nov. 25, 2018 Image Mitch Landrieu after receiving the 2018 John F. K..."
6,john,https://www.bbc.co.uk/news/world-46336355,Navigation Sections Home UK World selected Business Politics Tech Science Health Family & Education Entertainment & Arts Stories Video & Audio In Pictures Newsbeat Reality Check Special Reports Explainers The Reporters Have Your Say Disability World selected Africa Asia Australia Europe Latin Am...
7,john,https://www.cnn.com/2018/11/16/us/john-hinckley-can-move-out-of-mothers-house/index.html,Studios CNN Films HLN TV Schedule TV Shows A-Z CNNVR Coupons CNN Underscored -Explore -Wellness -Gadgets -Lifestyle CNN Store How To Watch VR Archives More… Photos Longform Investigations CNN profiles CNN Leadership CNN Newsletters Work for CNN U.S. International Español Arabic Set edition prefe...
8,john,https://www.nytimes.com/2018/11/22/opinion/sunday/thanksgiving-politics-family.html,"Sections SEARCH Skip to content Skip to site index Sunday Review Subscribe Log In Log In Today’s Paper Sunday Review | Of Monuments, Arguments, Vampires and Thanksgiving Opinion Supported by Of Monuments, Arguments, Vampires and Thanksgiving John Wayne, Brett Kavanaugh, my brothers Michael and K..."
9,john,https://www.cnn.com/videos/entertainment/2018/11/14/chrissy-teigen-award-emotions-john-legend-orig-vstan-bdk.cnn,"Find out what's happening in the world as it unfolds. JUST WATCHED Teigen, Legend cry 'sappy' tears at award show Replay More Videos ... MUST WATCH Teigen, Legend cry 'sappy' tears at award show Chrissy Teigen's speech while receiving Glamour's 2018 Women of the Year award turned ""sappy"" after h..."



Time taken: 33.0 seconds

Do you want to search again? [y/n] n

THANK YOU!, for searching with us!
